In [1]:
# можно до редьюса и группировки по reqid сделать промежуточную табличку и локально подебажить обход по дереву
# https://a.yandex-team.ru/arc_vcs/search/analytics_keymetrics/regular/direct_cabinet/baobab_to_events.py?rev=d49dbef6677082aa0c3bcb6ec5111506e601cc94#L255

In [2]:
import yt.wrapper as yt
import baobab
import datetime as dt

yt_client = yt.YtClient(proxy="hahn")
path = '//home/vipplanners/users/n-bar/OPERANALYTICS-1983/nile_test/2021-04-09T09:00:00["8066727188962238556"]'
table = yt_client.read_table(path, format=yt.JsonFormat(), raw=False)

records = list(table)
print "Строк: %s" % len(records)
records[0]

Строк: 698


{'HTTP_REFERER': 'https://direct.yandex.ru/dna/grid/campaigns?filter=%D0%A1%D1%82%D0%B0%D1%82%D1%83%D1%81%20%3D%20%D0%92%D1%81%D0%B5%2C%20%D0%BA%D1%80%D0%BE%D0%BC%D0%B5%20%D0%B0%D1%80%D1%85%D0%B8%D0%B2%D0%BD%D1%8B%D1%85%7Cdim%20%3D%20CPC&tags=&ulogin=kim-a-2015',
 'events': '%5B%7B%22data%22%3A%7B%22uid%22%3A%22319077082%22%2C%22yandexuid%22%3A%227918359461598019922%22%7D%2C%22event%22%3A%22visibility%22%2C%22type%22%3A%22hide%22%2C%22block-id%22%3A%22jx14bs%22%2C%22cts%22%3A1617949519186%2C%22trigger-event-id%22%3A%22kn9xd80a3%22%2C%22trigger-event-trusted%22%3Afalse%7D%5D',
 'experiments': [[349809, 53],
  [335013, 26],
  [267758, 89],
  [267755, 67],
  [352301, 9],
  [348335, 90],
  [330180, 33],
  [0, 0]],
 'icookie': '7918359461598019922',
 'ip': '94.25.168.108,94.25.168.108',
 'reqid': '8066727188962238556',
 'slots': '349809,0,53;335013,0,26;267758,0,89;267755,0,67;352301,0,9;348335,0,90;330180,0,33',
 'table': 'direct',
 'unixtime': '1617949527',
 'url': '//direct.yandex.ru',
 

In [3]:
import json
from urllib import unquote

json.loads(unquote(records[0]["events"]))

[{u'block-id': u'jx14bs',
  u'cts': 1617949519186,
  u'data': {u'uid': u'319077082', u'yandexuid': u'7918359461598019922'},
  u'event': u'visibility',
  u'trigger-event-id': u'kn9xd80a3',
  u'trigger-event-trusted': False,
  u'type': u'hide'}]

In [4]:
app_n = 0
shows = 0
clicks = 0
for r in records:
    json_events = json.loads(unquote(r["events"]))
    if 'tree' in json_events[0] and json_events[0]['tree']['name'] == 'app':
        app = json_events[0]
        app_n += 1
    if 'event' in json_events[0] and json_events[0]['event'] == 'show':
        shows += 1
    if 'event' in json_events[0] and json_events[0]['event'] == 'click':
        clicks += 1
app

{u'cts': 1617949381888,
 u'data': {u'uid': u'319077082', u'yandexuid': u'7918359461598019922'},
 u'event': u'show',
 u'tree': {u'attrs': {u'pageUrl': u'direct.yandex.ru/dna/grid/campaigns?filter=%D0%A1%D1%82%D0%B0%D1%82%D1%83%D1%81%20%3D%20%D0%92%D1%81%D0%B5%2C%20%D0%BA%D1%80%D0%BE%D0%BC%D0%B5%20%D0%B0%D1%80%D1%85%D0%B8%D0%B2%D0%BD%D1%8B%D1%85%7Cdim%20%3D%20CPM&ulogin=kim-a-2015',
   u'schema-ver': 0,
   u'service': u'direct',
   u'ui': u'desktop'},
  u'children': [{u'children': [{u'id': u'jx142', u'name': u'ServicesMenu'},
     {u'id': u'jx143', u'name': u'YandexLogo'},
     {u'id': u'jx144', u'name': u'DirectLogo'},
     {u'id': u'jx145', u'name': u'CommonWalletSettings'},
     {u'id': u'jx146', u'name': u'HeaderInfoBlock'},
     {u'id': u'jx147', u'name': u'FeedbackTooltip'},
     {u'id': u'jx148', u'name': u'User'}],
    u'id': u'jx141',
    u'name': u'Header'},
   {u'children': [{u'attrs': {u'id': u'expand'},
      u'id': u'jx14a',
      u'name': u'SidebarMenuItem'}],
    u'id': u

In [5]:
app_n, shows, clicks

(1, 1, 12)

In [6]:
# def reduce_get_event_content(groups):
# groups: [key|reqid, records]
# records: [rows|fields]
transport = baobab.TransportLevelParser()
trees_merger = baobab.common.TreesAccumulateMerger()
yandexuid = None
slots = None

for fields in records:
    fields["dtype"] = "iweb"
    # парсинг контекста рядом с событием
    ctx = transport.create_from_raw_redir_log_fields_dict(fields)
    if ctx is not None:
        trees_merger.add(ctx)
        yandexuid = ctx.uid if ctx.uid else yandexuid
        slots = fields["slots"] if "slots" in fields else slots

ctxs_merged = trees_merger.join()
print "Контекст транспорта для хита:", len(ctxs_merged.ctxs)
print "yandexuid:", yandexuid
print "slots:", slots

Контекст транспорта для хита: 617
yandexuid: 7918359461598019922
slots: 349809,0,53;335013,0,26;267758,0,89;267755,0,67;352301,0,9;348335,0,90;330180,0,33


In [7]:
# парсинг самих событий с раскладкой по деревьям
visitor = baobab.visitor.ShowIndexerVisitor()
parser = baobab.EventsParser()

for ctx in ctxs_merged.ctxs:
    parser.parse(ctx, visitor)

print "Деревьев:", len(visitor.baobab_tree_id_2_baobab_data)

Деревьев: 1


In [8]:
# склейка показов и кликов для одного дерева
def join_data(data):
    joiner = baobab.common.ShowAndClicksJoiner()
    if data.show is not None:
        joiner.set_show(data.show)
    for click in data.clicks:
        joiner.add_event(click)
    for tech in data.techs:
        joiner.add_event(tech)
    joiner.join()
    return joiner

joined_trees = [join_data(x) for x in visitor.baobab_tree_id_2_baobab_data.itervalues() if x]
print "Деревьев с подклееными кликами:", len(joined_trees)

Деревьев с подклееными кликами: 1


In [9]:
def get_path_from_root(block):
    blocks = []
    cur_block = block
    while cur_block is not None:
        block_type = cur_block.attrs.get('name', '')
        blocks.append(cur_block.name + (('.' + cur_block.attrs.get('id')) if 'id' in cur_block.attrs else ''))
        cur_block = cur_block.parent
    return list(reversed(blocks))


def get_attrs_from_root(block):
    blocks = []
    cur_block = block
    while cur_block is not None:
        blocks.append(cur_block.attrs)
        cur_block = cur_block.parent
    return list(reversed(blocks))



def merge_two_dicts(x, y):
    z = x.copy()
    z.update(y)
    return z


def get_path_info_from_root(block):
    report_paths = []
    block_paths = []
    attrs = []
    cur_block = block
    while cur_block is not None:
        report_paths.append(cur_block.attrs.get('id', cur_block.attrs.get('title', cur_block.name)))
        block_paths.append(cur_block.name)
        attrs.append(cur_block.attrs)
        cur_block = cur_block.parent

#     мердж от корня, потомки имеют приоритет по одинаковым ключам
    merged_attrs = reduce(lambda lhs, rhs: merge_two_dicts(lhs, rhs), reversed(attrs))

    return list(reversed(report_paths)), list(reversed(block_paths)), merged_attrs

In [10]:
tree = joined_trees[0]
all_events = tree.get_all_events()
print "Событий кроме показов:", len(all_events)

Событий кроме показов: 12


In [11]:
click = all_events[5]
# click.context._event_data.get('data')
click.context._event_data

{u'cts': 1617949662149,
 u'data': {u'type': u'allCheckboxClick'},
 u'event': u'click',
 u'event-id': u'kn9xgd6ta',
 u'id': u'jx14m3',
 u'service': u'direct'}

In [12]:
# корень для начала обхода дерева показов
root_block = tree.get_show().tree.root

In [13]:
get_path_from_root(root_block.first_child.first_child)

[u'app', u'Header', u'ServicesMenu']

In [14]:
get_attrs_from_root(root_block.first_child.first_child)

[{u'pageUrl': u'direct.yandex.ru/dna/grid/campaigns?filter=%D0%A1%D1%82%D0%B0%D1%82%D1%83%D1%81%20%3D%20%D0%92%D1%81%D0%B5%2C%20%D0%BA%D1%80%D0%BE%D0%BC%D0%B5%20%D0%B0%D1%80%D1%85%D0%B8%D0%B2%D0%BD%D1%8B%D1%85%7Cdim%20%3D%20CPM&ulogin=kim-a-2015',
  u'schema-ver': 0,
  u'service': u'direct',
  u'ui': u'desktop'},
 {},
 {}]

In [15]:
get_path_info_from_root(root_block.first_child.first_child)

([u'app', u'Header', u'ServicesMenu'],
 [u'app', u'Header', u'ServicesMenu'],
 {u'pageUrl': u'direct.yandex.ru/dna/grid/campaigns?filter=%D0%A1%D1%82%D0%B0%D1%82%D1%83%D1%81%20%3D%20%D0%92%D1%81%D0%B5%2C%20%D0%BA%D1%80%D0%BE%D0%BC%D0%B5%20%D0%B0%D1%80%D1%85%D0%B8%D0%B2%D0%BD%D1%8B%D1%85%7Cdim%20%3D%20CPM&ulogin=kim-a-2015',
  u'schema-ver': 0,
  u'service': u'direct',
  u'ui': u'desktop'})

In [16]:
def process_tree(joined_data):

    show = joined_data.get_show()
    root_block = show.tree.root
    block_shows = []
    block_clicks = dict()

    show_data =  show.context._event_data.get('data', dict())
    puid = show_data.get('uid')
    yandexuid = show_data.get('yandexuid')
    
    tree_time = show.context.server_time
    tree_date = dt.datetime.fromtimestamp(show.context.server_time).strftime("%Y-%m-%d")

    for block in baobab.common.dfs_iterator(root_block):
        show_path, block_path, show_attrs = get_path_info_from_root(block)
        page_url = show_attrs.get("pageUrl")

        block_shows.append({
            "report_path": show_path,
            "block_path": block_path,
            "page_url": page_url,
            "attrs": show_attrs,
            "data": show_data,
            "action": "show",
            "puid": puid,
            "yandexuid": yandexuid,
            "tree_time": tree_time,
            "tree_date": tree_date,
            "event_time": tree_time
        })

        for event in joined_data.get_events_by_block(block):
            if isinstance(event, baobab.common.event.Click):
                click_attrs = merge_two_dicts(show_attrs, event.context._event_data.get("attrs", dict()))
                click_data = merge_two_dicts(show_data, event.context._event_data.get('data', dict()))
                page_url = click_attrs.get("pageUrl", page_url)
                block_clicks[event.event_id] = {
                    "report_path": show_path,
                    "block_path": block_path,
                    "page_url": page_url,
                    "attrs": click_attrs,
                    "data": click_data,
                    "action": "click",
                    "puid": puid,
                    "yandexuid": yandexuid,
                    "tree_time": tree_time,
                    "tree_date": tree_date,
                    "event_time": event.context.server_time
                }

    return block_shows + block_clicks.values()

events = process_tree(joined_trees[0])
list(filter(lambda x: x["action"] == "click", events))

[{'action': 'click',
  'attrs': {u'pageUrl': u'direct.yandex.ru/dna/grid/campaigns?filter=%D0%A1%D1%82%D0%B0%D1%82%D1%83%D1%81%20%3D%20%D0%92%D1%81%D0%B5%2C%20%D0%BA%D1%80%D0%BE%D0%BC%D0%B5%20%D0%B0%D1%80%D1%85%D0%B8%D0%B2%D0%BD%D1%8B%D1%85%7Cdim%20%3D%20CPM&ulogin=kim-a-2015',
   u'schema-ver': 0,
   u'service': u'direct',
   u'ui': u'desktop'},
  'block_path': [u'app', u'ButtonDecorator'],
  'data': {u'uid': u'319077082', u'yandexuid': u'7918359461598019922'},
  'event_time': 1617949601,
  'page_url': u'direct.yandex.ru/dna/grid/campaigns?filter=%D0%A1%D1%82%D0%B0%D1%82%D1%83%D1%81%20%3D%20%D0%92%D1%81%D0%B5%2C%20%D0%BA%D1%80%D0%BE%D0%BC%D0%B5%20%D0%B0%D1%80%D1%85%D0%B8%D0%B2%D0%BD%D1%8B%D1%85%7Cdim%20%3D%20CPM&ulogin=kim-a-2015',
  'puid': u'319077082',
  'report_path': [u'app', u'ButtonDecorator'],
  'tree_date': '2021-04-09',
  'tree_time': 1617949385,
  'yandexuid': u'7918359461598019922'},
 {'action': 'click',
  'attrs': {u'pageUrl': u'direct.yandex.ru/dna/grid/campaigns?filter=

In [17]:
for tree in joined_trees:
    print tree.get_show().tree.root.name

app


In [18]:
click = tree.get_events_by_block(tree.get_show().tree.root.first_child.next_sibling)

In [19]:
tree.get_show().tree.get_block_by_id(click[click].block_id)

TypeError: list indices must be integers, not list